In [ ]:
from google.colab import drive
import os

# Check if the mount point exists and is not empty, then remove its contents
mountpoint = '/content/drive'
if os.path.exists(mountpoint) and os.path.isdir(mountpoint):
    if os.listdir(mountpoint):
        print(f"Removing contents of {mountpoint}...")
        for item in os.listdir(mountpoint):
            item_path = os.path.join(mountpoint, item)
            if os.path.isfile(item_path):
                os.remove(item_path)
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)

drive.mount(mountpoint)

Mounted at /content/drive


In [ ]:
import os
# Path to your dataset inside Google Drive
data_path = "/content/drive/MyDrive/solarPanelDefectDetection"
# Check folders
os.listdir(data_path)

['Physical-Damage',
 'Snow-Covered',
 'Dusty',
 'Electrical-damage',
 'Clean',
 'Bird-drop']

Binary classification into defective and non defective

In [ ]:
import os
import shutil
# Source folder where your 6 original folders are located
source_path = "/content/drive/MyDrive/solarPanelDefectDetection"
# Destination folder for grouped dataset
dest_path = "/content/drive/MyDrive/solarPanelDefectDetection_Binary"
os.makedirs(dest_path, exist_ok=True)
# Create the two main folders
os.makedirs(os.path.join(dest_path, "defective"), exist_ok=True)
os.makedirs(os.path.join(dest_path, "non_defective"), exist_ok=True)

In [ ]:
# Defective: Electrical-damage + Physical-Damage
for folder in ["Electrical-damage", "Physical-Damage"]:
    src = os.path.join(source_path, folder)
    for item_name in os.listdir(src):
        item_path = os.path.join(src, item_name)
        if os.path.isfile(item_path):
            shutil.copy(item_path, os.path.join(dest_path, "defective"))
# Non-defective: Bird-drop + Dusty + Clean + Snow-Covered
for folder in ["Bird-drop", "Dusty", "Clean", "Snow-Covered"]:
    src = os.path.join(source_path, folder)
    for item_name in os.listdir(src):
        item_path = os.path.join(src, item_name)
        if os.path.isfile(item_path):
            shutil.copy(item_path, os.path.join(dest_path, "non_defective"))
print("Files grouped successfully!")
print("Defective images:", len(os.listdir(os.path.join(dest_path, "defective"))))
print("Non-defective images:", len(os.listdir(os.path.join(dest_path, "non_defective"))))

Files grouped successfully!
Defective images: 173
Non-defective images: 698


Normal train test splitting

In [ ]:
!pip install split-folders
import splitfolders
splitfolders.ratio(
    dest_path,
    output="/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification",
    seed=42,
    ratio=(.7, .2, .1)  # 70% train, 20% val, 10% test
)

Copying files: 871 files [00:23, 37.02 files/s]


In [ ]:
import os
for root, dirs, files in os.walk("/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification"):
    level = root.replace("/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification", "").count(os.sep)
    indent = " " * 2 * level
    print(f"{indent}{os.path.basename(root)}/")

binaryClassification/
  train/
    defective/
    non_defective/
  val/
    defective/
    non_defective/
  test/
    defective/
    non_defective/


hii

In [ ]:
import os
base_dir = '/content/drive/MyDrive/solarPanelDefectDetection_Binary'
print(f"Counting images inside: {base_dir}\n")
for folder_name in sorted(os.listdir(base_dir)):
    folder_path = os.path.join(base_dir, folder_name)
    if os.path.isdir(folder_path):
        # Count all images recursively (in subfolders too)
        count = 0
        for root, _, files in os.walk(folder_path):
            count += sum(
                f.lower().endswith(('.jpg', '.jpeg', '.png'))
                for f in files
            )
        print(f"{folder_name}: {count} images")
print("\nImage count complete!")


Counting images inside: /content/drive/MyDrive/solarPanelDefectDetection_Binary

balancedClasses: 1380 images
batchesNonDefective: 697 images
binaryClassification: 869 images
defective: 172 images
non_defective: 697 images

Image count complete!


In [ ]:
import os
import shutil
import re
import math
source_dir = '/content/drive/MyDrive/solarPanelDefectDetection_Binary/non_defective'
batches_dir = '/content/drive/MyDrive/solarPanelDefectDetection_Binary/batchesNonDefective'
batch_size = 173  # target number of images per batch
os.makedirs(batches_dir, exist_ok=True)
# === Helper for natural sorting ===
def numeric_sort_key(filename):
    """Extract numbers from filenames like bird(12).jpg for natural sorting."""
    nums = re.findall(r'\d+', filename)
    return int(nums[-1]) if nums else 0
# === Collect & sort all images ===
all_images = [f for f in os.listdir(source_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
all_images.sort(key=numeric_sort_key)
total_images = len(all_images)
num_batches = math.ceil(total_images / batch_size)
print(f"Total images: {total_images}")
print(f"Creating {num_batches} batches of up to {batch_size} images each...\n")
for i in range(num_batches):
    batch_path = os.path.join(batches_dir, f'batch_{i+1}')
    os.makedirs(batch_path, exist_ok=True)
    start = i * batch_size
    # last batch takes all remaining images (handles any off-by-one)
    end = min((i + 1) * batch_size, total_images)
    batch_files = all_images[start:end]
    for f in batch_files:
        shutil.copy2(os.path.join(source_dir, f), os.path.join(batch_path, f))
    print(f"Batch_{i+1} created with {len(batch_files)} sorted images.")
count_check = sum(len(os.listdir(os.path.join(batches_dir, b))) for b in os.listdir(batches_dir))
print(f"\nVerification: {count_check}/{total_images} images successfully copied.\n")
print("All sorted batches created successfully!")

Total images: 697
Creating 5 batches of up to 173 images each...

Batch_1 created with 173 sorted images.
Batch_2 created with 173 sorted images.
Batch_3 created with 173 sorted images.
Batch_4 created with 173 sorted images.
Batch_5 created with 5 sorted images.

Verification: 697/697 images successfully copied.

All sorted batches created successfully!


In [ ]:
import os
batches_dir = '/content/drive/MyDrive/solarPanelDefectDetection_Binary/batchesNonDefective'
print("Number of images in each batch:\n")
total_count = 0
for batch_name in sorted(os.listdir(batches_dir)):
    batch_path = os.path.join(batches_dir, batch_name)
    if os.path.isdir(batch_path):
        num_images = len([f for f in os.listdir(batch_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
        print(f"{batch_name}: {num_images} images")
        total_count += num_images
print(f"\nTotal images across all batches: {total_count}")

Number of images in each batch:

batch_1: 173 images
batch_2: 173 images
batch_3: 173 images
batch_4: 173 images
batch_5: 5 images

Total images across all batches: 697


In [ ]:
import os
from collections import Counter
# Path where the 5 non_defective batches are stored
batches_dir = '/content/drive/MyDrive/solarPanelDefectDetection_Binary/batchesNonDefective'
# Store all filenames to check duplicates
all_files = []
print("Batch Summary:\n")
for batch_name in sorted(os.listdir(batches_dir)):
    batch_path = os.path.join(batches_dir, batch_name)
    if os.path.isdir(batch_path):
        files = [f for f in os.listdir(batch_path) if os.path.isfile(os.path.join(batch_path, f))]
        all_files.extend(files)
        print(f"{batch_name}: {len(files)} images")
# Check for duplicates
duplicates = [item for item, count in Counter(all_files).items() if count > 1]
print("\nDuplicate Check:")
if duplicates:
    print(f"Found {len(duplicates)} duplicate files across batches:")
    for dup in duplicates[:10]:
        print(f"   - {dup}")
else:
    print("No duplicate filenames found across batches!")

print("\nBatch verification complete.")

Batch Summary:

batch_1: 173 images
batch_2: 173 images
batch_3: 173 images
batch_4: 173 images
batch_5: 5 images

Duplicate Check:
No duplicate filenames found across batches!

Batch verification complete.


In [ ]:
import os
import shutil
import re
base_dir = '/content/drive/MyDrive/solarPanelDefectDetection_Binary'
defective_dir = os.path.join(base_dir, 'defective')
batches_dir = os.path.join(base_dir, 'batchesNonDefective')
# === Helper functions ===
def numeric_sort_key(filename):
    """Sorts filenames naturally like bird(2).jpg < bird(10).jpg"""
    nums = re.findall(r'\d+', filename)
    return int(nums[-1]) if nums else 0
def get_sorted_images(path):
    """Get all sorted image filenames from a folder."""
    return sorted(
        [f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))],
        key=numeric_sort_key
    )
def copy_files(files, src, dst):
    """Safely copy files from src to dst."""
    os.makedirs(dst, exist_ok=True)
    for f in files:
        shutil.copy2(os.path.join(src, f), os.path.join(dst, f))
# === Load defective images ===
defective_imgs = get_sorted_images(defective_dir)
n_def = len(defective_imgs)
n_train = int(0.7 * n_def)
n_val = int(0.2 * n_def)
n_test = n_def - n_train - n_val
print(f"Defective total: {n_def} → Train:{n_train}, Val:{n_val}, Test:{n_test}\n")
# === Process each non-defective batch ===
for batch_name in sorted(os.listdir(batches_dir)):
    batch_path = os.path.join(batches_dir, batch_name)
    if not os.path.isdir(batch_path):
        continue
    nondef_imgs = get_sorted_images(batch_path)
    if len(nondef_imgs) < n_def:
        print(f"Skipping {batch_name}: only {len(nondef_imgs)} images (need {n_def})")
        continue
    # Take only the first n_def images to balance
    nondef_imgs = nondef_imgs[:n_def]
    # Split same way
    non_train = nondef_imgs[:n_train]
    non_val   = nondef_imgs[n_train:n_train+n_val]
    non_test  = nondef_imgs[n_train+n_val:]
    def_train = defective_imgs[:n_train]
    def_val   = defective_imgs[n_train:n_train+n_val]
    def_test  = defective_imgs[n_train+n_val:]
    # === Create output path for this batch ===
    output_dir = os.path.join(base_dir, f'SplitBalanced_{batch_name}')
    for split in ['train', 'val', 'test']:
        for cls in ['defective', 'non_defective']:
            os.makedirs(os.path.join(output_dir, split, cls), exist_ok=True)
    # === Copy files ===
    copy_files(def_train, defective_dir, os.path.join(output_dir, 'train', 'defective'))
    copy_files(def_val, defective_dir, os.path.join(output_dir, 'val', 'defective'))
    copy_files(def_test, defective_dir, os.path.join(output_dir, 'test', 'defective'))
    copy_files(non_train, batch_path, os.path.join(output_dir, 'train', 'non_defective'))
    copy_files(non_val, batch_path, os.path.join(output_dir, 'val', 'non_defective'))
    copy_files(non_test, batch_path, os.path.join(output_dir, 'test', 'non_defective'))
    print(f"Balanced split created for {batch_name}")
    print(f"Saved in: {output_dir}\n")
print("All 4 balanced datasets created successfully!")


Defective total: 172 → Train:120, Val:34, Test:18

Balanced split created for batch_1
Saved in: /content/drive/MyDrive/solarPanelDefectDetection_Binary/SplitBalanced_batch_1

Balanced split created for batch_2
Saved in: /content/drive/MyDrive/solarPanelDefectDetection_Binary/SplitBalanced_batch_2

Balanced split created for batch_3
Saved in: /content/drive/MyDrive/solarPanelDefectDetection_Binary/SplitBalanced_batch_3

Balanced split created for batch_4
Saved in: /content/drive/MyDrive/solarPanelDefectDetection_Binary/SplitBalanced_batch_4

Skipping batch_5: only 5 images (need 172)
All 4 balanced datasets created successfully!


In [ ]:
import os
import shutil
base_dir = '/content/drive/MyDrive/solarPanelDefectDetection_Binary'
target_dir = os.path.join(base_dir, 'balancedClasses')
os.makedirs(target_dir, exist_ok=True)
# Move all SplitBalanced_* folders into BalancedClasses
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if folder.startswith('SplitBalanced_') and os.path.isdir(folder_path):
        dest_path = os.path.join(target_dir, folder)
        print(f"Moving {folder} → balancedClasses/")
        shutil.move(folder_path, dest_path)
print("\nAll SplitBalanced_* folders moved successfully!")
print(f"New location: {target_dir}")


Moving SplitBalanced_batch_1 → balancedClasses/
Moving SplitBalanced_batch_2 → balancedClasses/
Moving SplitBalanced_batch_3 → balancedClasses/
Moving SplitBalanced_batch_4 → balancedClasses/

All SplitBalanced_* folders moved successfully!
New location: /content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses


In [ ]:
import os
folder = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses/SplitBalanced_batch_1/train/defective"
count = 0
for fname in os.listdir(folder):
    if fname.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")):
        count += 1
print("Number of images:", count)

Number of images: 120


In [ ]:
!ls "/content/drive/MyDrive/"

'Colab Notebooks'	     solarPanelDefectDetection_Binary
 solarPanelDefectDetection   solarPanelDefectDetection_Reorganized


In [ ]:
import os
data_dir = "/content/drive/MyDrive/solarPanelDefectDetection_Binary"
for root, dirs, files in os.walk(data_dir):
    level = root.replace(data_dir, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    print(f"{subindent}{len(files)} files")


solarPanelDefectDetection_Binary/
    0 files
    defective/
        173 files
    non_defective/
        698 files
    binaryClassification/
        0 files
        train/
            0 files
            defective/
                121 files
            non_defective/
                488 files
        val/
            0 files
            defective/
                34 files
            non_defective/
                139 files
        test/
            0 files
            defective/
                18 files
            non_defective/
                71 files
        model_results/
            11 files
    batchesNonDefective/
        0 files
        batch_1/
            173 files
        batch_2/
            173 files
        batch_3/
            173 files
        batch_4/
            173 files
        batch_5/
            5 files
    balancedClasses/
        0 files
        SplitBalanced_batch_1/
            0 files
            train/
                0 files
                defective/
 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
train_dir = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/train"
val_dir = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/val"
test_dir = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/test"

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
val_gen = ImageDataGenerator(rescale=1./255)
train_data = train_gen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')
val_data = val_gen.flow_from_directory(val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')
test_data = val_gen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', shuffle=False)


Found 607 images belonging to 2 classes.
Found 173 images belonging to 2 classes.
Found 89 images belonging to 2 classes.


In [ ]:
!ls '/content/drive/MyDrive/solarPanelDefectDetection_Binary'

balancedClasses      binaryClassification  non_defective
batchesNonDefective  defective


Double Binary Classification

In [ ]:
import os
import shutil

# === SOURCE: your raw dataset ===
source_dir = '/content/drive/MyDrive/solarPanelDefectDetection'

# === DESTINATION: organized dataset ===
target_dir = '/content/drive/MyDrive/solarPanelDefectDetection_Reorganized'

# Define final folder paths
defective_dir = os.path.join(target_dir, 'Defective')
non_defective_dir = os.path.join(target_dir, 'Non-Defective')
non_clean_dir = os.path.join(non_defective_dir, 'Non-Clean')

# Create directories
os.makedirs(defective_dir, exist_ok=True)
os.makedirs(non_defective_dir, exist_ok=True)
os.makedirs(non_clean_dir, exist_ok=True)

# Mapping from your original folder names → destination folders
move_map = {
    'Electrical-damage': os.path.join(defective_dir, 'Electrical-damage'),
    'Physical-Damage': os.path.join(defective_dir, 'Physical-Damage'),
    'Clean': os.path.join(non_defective_dir, 'Clean'),
    'Snow-Covered': os.path.join(non_clean_dir, 'Snow-Covered'),
    'Bird-drop': os.path.join(non_clean_dir, 'Bird-drop'),
    'Dusty': os.path.join(non_clean_dir, 'Dusty'),
}

# Copy all matching folders
for folder_name, dest_path in move_map.items():
    src_path = os.path.join(source_dir, folder_name)
    if os.path.exists(src_path):
        print(f"📂 Copying {folder_name} → {dest_path}")
        shutil.copytree(src_path, dest_path, dirs_exist_ok=True)
    else:
        print(f"⚠️ Folder not found: {src_path}")

print("\n✅ Dataset successfully reorganized!")
print(f"📁 Saved in: {target_dir}")


📂 Copying Electrical-damage → /content/drive/MyDrive/solarPanelDefectDetection_Reorganized/Defective/Electrical-damage
📂 Copying Physical-Damage → /content/drive/MyDrive/solarPanelDefectDetection_Reorganized/Defective/Physical-Damage
📂 Copying Clean → /content/drive/MyDrive/solarPanelDefectDetection_Reorganized/Non-Defective/Clean
📂 Copying Snow-Covered → /content/drive/MyDrive/solarPanelDefectDetection_Reorganized/Non-Defective/Non-Clean/Snow-Covered
📂 Copying Bird-drop → /content/drive/MyDrive/solarPanelDefectDetection_Reorganized/Non-Defective/Non-Clean/Bird-drop
📂 Copying Dusty → /content/drive/MyDrive/solarPanelDefectDetection_Reorganized/Non-Defective/Non-Clean/Dusty

✅ Dataset successfully reorganized!
📁 Saved in: /content/drive/MyDrive/solarPanelDefectDetection_Reorganized


In [ ]:
!ls -R '/content/drive/MyDrive/solarPanelDefectDetection_Reorganized'


/content/drive/MyDrive/solarPanelDefectDetection_Reorganized:
Defective  Non-Defective

/content/drive/MyDrive/solarPanelDefectDetection_Reorganized/Defective:
Electrical-damage  Physical-Damage

/content/drive/MyDrive/solarPanelDefectDetection_Reorganized/Defective/Electrical-damage:
 desktop.ini	       'Electrical (3).JPG'   'Electrical (6).jpg'
'Electrical (10).jpg'  'Electrical (3).png'   'Electrical (70).jpg'
'Electrical (11).jpg'  'Electrical (40).jpg'  'Electrical (71).jpg'
'Electrical (12).jpg'  'Electrical (41).jpg'  'Electrical (72).jpg'
'Electrical (13).jpg'  'Electrical (42).jpg'  'Electrical (73).JPG'
'Electrical (14).jpg'  'Electrical (43).jpg'  'Electrical (74).jpg'
'Electrical (15).jpg'  'Electrical (44).jpg'  'Electrical (75).jpg'
'Electrical (16).jpg'  'Electrical (45).jpg'  'Electrical (76).jpg'
'Electrical (17).jpg'  'Electrical (46).jpg'  'Electrical (77).jpg'
'Electrical (18).jpg'  'Electrical (47).jpg'  'Electrical (78).jpg'
'Electrical (19).jpg'  'Electrical (48

In [ ]:
import os
import shutil
import random

# === Paths ===
dataset_dir = '/content/drive/MyDrive/solarPanelDefectDetection_Reorganized'
split_dir   = os.path.join(dataset_dir, 'Classification')

# === Split ratios ===
train_ratio, val_ratio, test_ratio = 0.7, 0.2, 0.1
random.seed(42)

# === Valid image formats ===
image_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif')

# === Helper functions ===
def is_image_file(f):
    return f.lower().endswith(image_exts)

def make_dir(p):
    os.makedirs(p, exist_ok=True)

def copy_files(files, src, dst):
    for f in files:
        shutil.copy2(os.path.join(src, f), os.path.join(dst, f))

# === Start ===
print("🚀 Starting dataset split...\n")

# Remove old split folder if it exists
if os.path.exists(split_dir):
    print("🧹 Removing old Classification folder...")
    shutil.rmtree(split_dir)
make_dir(split_dir)

# Automatically find every category folder (Defective, Non-Defective)
for category in os.listdir(dataset_dir):
    category_path = os.path.join(dataset_dir, category)
    if not os.path.isdir(category_path) or category == 'Classification':
        continue

    print(f"\n📂 Processing category: {category}")

    for root, _, files in os.walk(category_path):
        image_files = [f for f in files if is_image_file(f)]
        if not image_files:
            continue

        rel_path = os.path.relpath(root, dataset_dir)
        n_total = len(image_files)
        random.shuffle(image_files)
        n_train = int(train_ratio * n_total)
        n_val = int(val_ratio * n_total)

        train_files = image_files[:n_train]
        val_files   = image_files[n_train:n_train + n_val]
        test_files  = image_files[n_train + n_val:]

        # Create split dirs
        for split in ['train', 'val', 'test']:
            make_dir(os.path.join(split_dir, split, rel_path))

        # Copy
        copy_files(train_files, root, os.path.join(split_dir, 'train', rel_path))
        copy_files(val_files, root, os.path.join(split_dir, 'val', rel_path))
        copy_files(test_files, root, os.path.join(split_dir, 'test', rel_path))

        print(f"   {rel_path}: {n_total} images → "
              f"Train:{len(train_files)} | Val:{len(val_files)} | Test:{len(test_files)}")

print("\n✅ Dataset successfully split!")
print(f"📁 Split dataset saved in: {split_dir}")

🚀 Starting dataset split...


📂 Processing category: Defective
   Defective/Electrical-damage: 103 images → Train:72 | Val:20 | Test:11
   Defective/Physical-Damage: 69 images → Train:48 | Val:13 | Test:8

📂 Processing category: Non-Defective
   Non-Defective/Non-Clean/Snow-Covered: 123 images → Train:86 | Val:24 | Test:13
   Non-Defective/Non-Clean/Bird-drop: 191 images → Train:133 | Val:38 | Test:20
   Non-Defective/Non-Clean/Bird-drop/New: 16 images → Train:11 | Val:3 | Test:2
   Non-Defective/Non-Clean/Dusty: 190 images → Train:133 | Val:38 | Test:19
   Non-Defective/Clean: 193 images → Train:135 | Val:38 | Test:20

✅ Dataset successfully split!
📁 Split dataset saved in: /content/drive/MyDrive/solarPanelDefectDetection_Reorganized/Classification


Normal Classification 6 classes

In [ ]:
import os
import shutil
import random
# === PATHS ===
source_dir = '/content/drive/MyDrive/solarPanelDefectDetection'  # original dataset
split_dir  = '/content/drive/MyDrive/solarPanelDefectDetection/Classification'  # where to save split data
# === SPLIT RATIOS ===
train_ratio, val_ratio, test_ratio = 0.7, 0.2, 0.1
random.seed(42)
# === VALID IMAGE EXTENSIONS ===
image_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif')
# === HELPERS ===
def is_image_file(f):
    return f.lower().endswith(image_exts)
def make_dir(path):
    os.makedirs(path, exist_ok=True)
def copy_files(file_list, src_folder, dest_folder):
    for f in file_list:
        shutil.copy2(os.path.join(src_folder, f), os.path.join(dest_folder, f))
# === CLEAN OLD SPLIT (optional) ===
if os.path.exists(split_dir):
    print("Removing old split directory...")
    shutil.rmtree(split_dir)
make_dir(split_dir)
print("Starting dataset split...\n")
# === SPLIT EACH CLASS FOLDER ===
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    if not os.path.isdir(class_path):
        continue
    # Get all image files
    images = [f for f in os.listdir(class_path) if is_image_file(f)]
    if not images:
        print(f"No images found in {class_name}, skipping.")
        continue
    # Shuffle and split
    random.shuffle(images)
    n_total = len(images)
    n_train = int(train_ratio * n_total)
    n_val   = int(val_ratio * n_total)
    train_files = images[:n_train]
    val_files   = images[n_train:n_train + n_val]
    test_files  = images[n_train + n_val:]
    # Create folders for each phase
    for phase in ['train', 'val', 'test']:
        make_dir(os.path.join(split_dir, phase, class_name))
    # Copy images
    copy_files(train_files, class_path, os.path.join(split_dir, 'train', class_name))
    copy_files(val_files,   class_path, os.path.join(split_dir, 'val', class_name))
    copy_files(test_files,  class_path, os.path.join(split_dir, 'test', class_name))
    print(f"{class_name}: {n_total} images → "
          f"Train:{len(train_files)} | Val:{len(val_files)} | Test:{len(test_files)}")
print("\nDataset split complete!")
print(f"Split saved in: {split_dir}")


Starting dataset split...

Physical-Damage: 69 images → Train:48 | Val:13 | Test:8
Snow-Covered: 123 images → Train:86 | Val:24 | Test:13
Dusty: 190 images → Train:133 | Val:38 | Test:19
Electrical-damage: 103 images → Train:72 | Val:20 | Test:11
Clean: 193 images → Train:135 | Val:38 | Test:20
Bird-drop: 191 images → Train:133 | Val:38 | Test:20
No images found in Classification, skipping.

Dataset split complete!
Split saved in: /content/drive/MyDrive/solarPanelDefectDetection/Classification


Balancing classes

In [ ]:
import os
import shutil
import random

# === PATHS ===
split_dir = '/content/drive/MyDrive/solarPanelDefectDetection/Classification'
balanced_dir = '/content/drive/MyDrive/solarPanelDefectDetection/BalancedClasses'
target_counts = {'train': 48, 'val': 13, 'test': 8}
image_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif')
def is_image_file(f):
    return f.lower().endswith(image_exts)
def make_dir(p):
    os.makedirs(p, exist_ok=True)
def copy_files(files, src_folder, dest_folder):
    for f in files:
        shutil.copy2(os.path.join(src_folder, f), os.path.join(dest_folder, f))
if os.path.exists(balanced_dir):
    print("Removing old balanced dataset...")
    shutil.rmtree(balanced_dir)
make_dir(balanced_dir)
print("Creating balanced dataset...\n")
for phase in ['train', 'val', 'test']:
    phase_dir = os.path.join(split_dir, phase)
    classes = [d for d in os.listdir(phase_dir) if os.path.isdir(os.path.join(phase_dir, d))]
    for class_name in classes:
        src_class_dir = os.path.join(phase_dir, class_name)
        dest_class_dir = os.path.join(balanced_dir, phase, class_name)
        make_dir(dest_class_dir)
        all_imgs = [f for f in os.listdir(src_class_dir) if is_image_file(f)]
        random.shuffle(all_imgs)
        n_target = target_counts[phase]
        if len(all_imgs) < n_target:
            print(f"{class_name} ({phase}): only {len(all_imgs)} images, cannot sample {n_target}. Using all available.")
            selected_imgs = all_imgs
        else:
            selected_imgs = all_imgs[:n_target]
        copy_files(selected_imgs, src_class_dir, dest_class_dir)
        print(f"{phase}/{class_name}: selected {len(selected_imgs)} images")
print("\nBalanced dataset created successfully!")
print(f"Saved in: {balanced_dir}")

Creating balanced dataset...

train/Physical-Damage: selected 48 images
train/Snow-Covered: selected 48 images
train/Dusty: selected 48 images
train/Electrical-damage: selected 48 images
train/Clean: selected 48 images
train/Bird-drop: selected 48 images
val/Physical-Damage: selected 13 images
val/Snow-Covered: selected 13 images
val/Dusty: selected 13 images
val/Electrical-damage: selected 13 images
val/Clean: selected 13 images
val/Bird-drop: selected 13 images
test/Physical-Damage: selected 8 images
test/Snow-Covered: selected 8 images
test/Dusty: selected 8 images
test/Electrical-damage: selected 8 images
test/Clean: selected 8 images
test/Bird-drop: selected 8 images

Balanced dataset created successfully!
Saved in: /content/drive/MyDrive/solarPanelDefectDetection/BalancedClasses


In [ ]:
for phase in ['train', 'val', 'test']:
    print(f"\n{phase.upper()}")
    phase_dir = os.path.join(balanced_dir, phase)
    for cls in os.listdir(phase_dir):
        count = len(os.listdir(os.path.join(phase_dir, cls)))
        print(f"{cls:20s}: {count}")



TRAIN
Physical-Damage     : 48
Snow-Covered        : 48
Dusty               : 48
Electrical-damage   : 48
Clean               : 48
Bird-drop           : 48

VAL
Physical-Damage     : 13
Snow-Covered        : 13
Dusty               : 13
Electrical-damage   : 13
Clean               : 13
Bird-drop           : 13

TEST
Physical-Damage     : 8
Snow-Covered        : 8
Dusty               : 8
Electrical-damage   : 8
Clean               : 8
Bird-drop           : 8
